In [1]:
# Leer el streaming desde Kafka

retrasosStreamingDF = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "micluster-w-0:9092,micluster-w-1:9092") \
    .option("subscribe", "retrasos") \
    .load()

In [ ]:
#Leer la estructura de Streaming
retrasosStreamingDF.printSchema()

Todos los campos son creadas automáticamente por Spark cuando leemos de Kafka. De ellas, la que nos interesa es `value` que contiene propiamente el mensaje de Kafka

In [ ]:

#DAR FORMATO DE LECTURA AL VALOR VALUE

from pyspark.sql.functions import col, from_json
from pyspark.sql.types import StructType, StructField, StringType, IntegerType,DoubleType

# Definir el esquema del JSON que estás esperando en el campo VALUE
json_schema = StructType([
    StructField("dest", StringType(), True),
    StructField("arr_delay", DoubleType(), True)
])

# Convertir el campo `value` a string y luego parsear el JSON
parsedDF = retrasosStreamingDF.selectExpr("CAST(value AS STRING) as json_value") \
    .select(from_json(col("json_value"), json_schema).alias("data")) \
    .select("data.*")


# Mostrar el resultado en la consola
# Modo Append para verificar los mensajes que llegan
parsedDF.writeStream \
    .format("console") \
    .outputMode("append") \
    .start()

In [ ]:

#Para Guardar se sobreescribirá en un archivo parquet dentro de HFDS
parsedDF.writeStream \
    .format("parquet") \
    .option("path", "/resultado/miparquet_stream") \
    .option("checkpointLocation", "/resultado/checkpoint") \
    .outputMode("append") \
    .start()